# AprilTag Detection Testing Notebook

This notebook allows you to test different AprilTag detection methods and configurations on image files.


## 1. Setup and Imports


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add the module path
sys.path.insert(0, str(Path.cwd() / 'det_loc'))

# Import apriltag
try:
    import apriltag
except ImportError:
    print("Installing apriltag...")
    %pip install apriltag
    import apriltag

# Import the TagDetector
from det_loc.utils.tag_detector import TagDetector, draw_tag_detections

print("✓ All imports successful")


## 2. Helper Functions


In [ ]:
def load_image(image_path):
    """Load an image from file path."""
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    return img

def display_image(img, title="Image", figsize=(12, 8)):
    """Display a BGR image using matplotlib."""
    plt.figure(figsize=figsize)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def display_comparison(images, titles, figsize=(18, 6)):
    """Display multiple images side by side."""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_title(title)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

def print_detections(detections):
    """Print detection details."""
    print(f"\nFound {len(detections)} tags:")
    for det in detections:
        print(f"  - Tag ID: {det['id']}, Hamming: {det['hamming']}, "
              f"Center: ({det['center'][0]:.1f}, {det['center'][1]:.1f})")

print("✓ Helper functions loaded")


## 3. Initialize AprilTag Detector


In [ ]:
# Create the apriltag detector
options = apriltag.DetectorOptions(
    families='tag36h11',
    border=1,
    nthreads=4,
    quad_decimate=1.0,
    quad_blur=0.0,
    refine_edges=True,
    refine_decode=False,
    refine_pose=False,
    debug=False,
    quad_contours=True
)

detector = apriltag.Detector(options)
print("✓ AprilTag detector initialized")


## 4. Load Test Image

**Update the path below to point to your test image:**


In [ ]:
# TODO: Update this path to your test image
image_path = "path/to/your/test_image.jpg"

# Load the image
test_image = load_image(image_path)
print(f"Image loaded: {test_image.shape[1]}x{test_image.shape[0]} pixels")

# Display original image
display_image(test_image, "Original Image")


## 5. Test Different Detection Configurations

### 5.1 Test Different Scale Combinations


In [ ]:
# Test different scale configurations
scale_configs = [
    ([1.0], "Single scale: 1.0"),
    ([1.5, 2.0], "Two scales: 1.5, 2.0"),
    ([1.5, 2.0, 2.5], "Three scales: 1.5, 2.0, 2.5"),
    ([1.0, 1.5, 2.0, 2.5, 3.0], "Five scales: 1.0-3.0"),
]

results = []
for scales, label in scale_configs:
    config = {
        "scales": scales,
        "use_enhanced_processing": True,
        "draw_detections": True,
    }
    
    tag_detector = TagDetector(detector, config)
    result_img, detections = tag_detector.detect(test_image.copy())
    
    results.append((result_img, f"{label}\n({len(detections)} tags)"))
    print_detections(detections)

# Display all results
display_comparison(
    [r[0] for r in results],
    [r[1] for r in results],
    figsize=(20, 5)
)


### 5.2 Compare Enhanced vs. Standard Processing


In [ ]:
# Standard processing (no enhancement)
config_standard = {
    "scales": [1.5, 2.0, 2.5],
    "use_enhanced_processing": False,
    "draw_detections": True,
}

tag_detector_std = TagDetector(detector, config_standard)
result_std, detections_std = tag_detector_std.detect(test_image.copy())

print("Standard Processing:")
print_detections(detections_std)

# Enhanced processing
config_enhanced = {
    "scales": [1.5, 2.0, 2.5],
    "use_enhanced_processing": True,
    "draw_detections": True,
}

tag_detector_enh = TagDetector(detector, config_enhanced)
result_enh, detections_enh = tag_detector_enh.detect(test_image.copy())

print("\nEnhanced Processing:")
print_detections(detections_enh)

# Compare side by side
display_comparison(
    [result_std, result_enh],
    [f"Standard ({len(detections_std)} tags)", f"Enhanced ({len(detections_enh)} tags)"],
    figsize=(16, 6)
)


### 5.3 Test Different Duplicate Thresholds


In [ ]:
# Test different duplicate removal thresholds
thresholds = [10, 20, 50]

threshold_results = []
for thresh in thresholds:
    config = {
        "scales": [1.5, 2.0, 2.5],
        "use_enhanced_processing": True,
        "duplicate_threshold": thresh,
        "draw_detections": True,
    }
    
    tag_detector = TagDetector(detector, config)
    result_img, detections = tag_detector.detect(test_image.copy())
    
    threshold_results.append((result_img, f"Threshold: {thresh}\n({len(detections)} tags)"))
    print(f"\nThreshold {thresh}:")
    print_detections(detections)

# Display comparison
display_comparison(
    [r[0] for r in threshold_results],
    [r[1] for r in threshold_results],
    figsize=(18, 6)
)


## 6. Custom Configuration Testing

Experiment with your own configurations:


In [ ]:
# Custom configuration - modify these parameters
custom_config = {
    "scales": [1.5, 2.0, 2.5],  # Try different scales
    "use_enhanced_processing": True,  # Toggle enhancement
    "duplicate_threshold": 20,  # Adjust duplicate detection sensitivity
    "draw_detections": True,
}

custom_detector = TagDetector(detector, custom_config)
custom_result, custom_detections = custom_detector.detect(test_image.copy())

print("Custom Configuration Results:")
print_detections(custom_detections)

display_image(custom_result, "Custom Configuration Results")


## 7. Batch Processing Multiple Images


In [ ]:
# Process multiple images from a directory
# TODO: Update this to your images directory
image_directory = "path/to/your/images/"

# Get all image files
image_dir = Path(image_directory)
image_files = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))

print(f"Found {len(image_files)} images")

# Configure detector
batch_config = {
    "scales": [1.5, 2.0, 2.5],
    "use_enhanced_processing": True,
    "draw_detections": True,
}

batch_detector = TagDetector(detector, batch_config)

# Process each image
for img_file in image_files[:5]:  # Process first 5 images
    print(f"\n{'='*60}")
    print(f"Processing: {img_file.name}")
    print('='*60)
    
    img = load_image(img_file)
    result, detections = batch_detector.detect(img)
    
    print_detections(detections)
    display_image(result, f"Results: {img_file.name}")


## 8. Analyze Detection Quality


In [ ]:
# Analyze detection quality metrics
config = {
    "scales": [1.5, 2.0, 2.5],
    "use_enhanced_processing": True,
    "draw_detections": False,  # Don't draw for analysis
}

analyzer = TagDetector(detector, config)
_, detections = analyzer.detect(test_image.copy())

if len(detections) > 0:
    # Extract metrics
    tag_ids = [det['id'] for det in detections]
    hamming_scores = [det['hamming'] for det in detections]
    
    # Plot hamming scores
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(detections)), hamming_scores)
    plt.xlabel('Detection Index')
    plt.ylabel('Hamming Distance')
    plt.title('Detection Quality (Lower is Better)')
    plt.xticks(range(len(detections)), [f"ID {tid}" for tid in tag_ids])
    plt.tight_layout()
    plt.show()
    
    print(f"\nQuality Statistics:")
    print(f"  Average Hamming: {np.mean(hamming_scores):.2f}")
    print(f"  Best (min): {np.min(hamming_scores)}")
    print(f"  Worst (max): {np.max(hamming_scores)}")
else:
    print("No detections found for analysis")


## 9. Save Results


In [ ]:
# Save annotated image
output_path = "apriltag_detection_result.jpg"

config = {
    "scales": [1.5, 2.0, 2.5],
    "use_enhanced_processing": True,
    "draw_detections": True,
}

save_detector = TagDetector(detector, config)
result_to_save, final_detections = save_detector.detect(test_image.copy())

cv2.imwrite(output_path, result_to_save)
print(f"✓ Results saved to: {output_path}")
print(f"  Detected {len(final_detections)} tags")
